In [1]:
from ortools.sat.python import cp_model
import pandas as pd

In [38]:
#df.drop(['B', 'C'], axis=1)

In [ ]:
#Get availability
#requests = pd.read_excel('C:\\Users\\ryder\\Desktop\\ChurchesOpti\\Churchesoptimization.xlsx')
requests = requests.fillna(1)
requests = requests.drop(['Week'], axis = 1)

In [2]:
fakerequests = [[0,1,0,1], [1,1,1,1], [0,1,1,0], [1,0,0,1], [1,1,1,1], [1,0,0,1]]

In [3]:
fakerequests

[[0, 1, 0, 1],
 [1, 1, 1, 1],
 [0, 1, 1, 0],
 [1, 0, 0, 1],
 [1, 1, 1, 1],
 [1, 0, 0, 1]]

In [4]:
#Church 0 - 9 are real churches, 10-12 are dummy values to assign OPEN to
num_churches = 6
num_weeks = 4
max_assignments = 4
all_churches = range(num_churches)
all_weeks = range(num_weeks)

In [43]:
#requests.values.tolist()

In [5]:
model = cp_model.CpModel()

In [6]:
#Creates empty schedule
schedule = {}
for churches in all_churches:
    for weeks in all_weeks:
            schedule[(churches, weeks)] = model.NewBoolVar(f"church_{churches} week_{weeks}")

In [7]:
#Condition 1: Only one church can be assigned per week.
for weeks in all_weeks:
        model.AddExactlyOne(schedule[(churches, weeks)] for churches in all_churches)

In [8]:
#Condition 2: Each church can only be assigned x times per year
for church in all_churches:
    churchassigned = []
    for week in all_weeks:
        churchassigned.append(schedule[church, week])
    model.Add(sum(churchassigned) <= max_assignments)
    model.Add(sum(churchassigned) >= 0)

In [9]:
# pylint: disable=g-complex-comprehension
model.Maximize(
    sum(
        fakerequests[churches][weeks] * schedule[(churches, weeks)]
        for churches in all_churches
        for weeks in all_weeks
    )
)

#Can only be scheduled 1 day in a rolling 8 weeks Experimental
max_days_in_8weeks = 1

for weeks in all weeks:
    sum()    


for each 8 week window, sum of a given churches assignments = 1
    

In [29]:
for church in all_churches:
    for week in all_weeks[0:len(all_weeks)-3]:
        rolling4weeks = [schedule[church, all_weeks[week+3]]]
    #Need to append something to rolling4weeks here
    print(rolling4weeks)
        #Can only be scheduled 1 day in a rolling 4 weeks Experimental
        #sum_var = model.NewIntVar(0, 1, '')
        # This adds the hard constraints on the sum.
        #model.Add(sum_var == sum(works))
            #'weekly_sum_constraint(employee %i, shift %i, week %i)' %
            #(church, week))
        #obj_int_vars.extend(variables)
        #obj_int_coeffs.extend(coeffs)


#Can only be scheduled 1 day in a rolling 8 weeks Experimental
#sum_var = model.NewIntVar(hard_min, hard_max, '')
# This adds the hard constraints on the sum.
#model.Add(sum_var == sum(works))

KeyError: (0, range(0, 3))

In [28]:
[schedule[0, all_weeks[0+3]]]

[church_0 week_3(0..1)]

In [64]:
#Can only be scheduled 1 day in a rolling 8 weeks Experimental
mingap = 2
for church in all_churches:
    churchassigned = []
    for week in all_weeks:
        churchassigned.append(schedule[church, week])
    model.Add(sum(churchassigned) <= 1)
   

In [65]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = False

In [66]:
class ChurchesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, schedule, num_churches, num_weeks, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._schedule = schedule
        self._num_churches = num_churches
        self._num_weeks = num_weeks
        #self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for weeks in range(self._num_weeks):
            print(f"Week {weeks}")
            for churches in range(self._num_churches):
                is_scheduled = False
                #for weeks in range(self._num_weeks):
                if self.Value(self._schedule[(churches, weeks)]):
                    is_scheduled = True
                    print(f"  Church {churches} is scheduled on week {weeks}")
                if not is_scheduled:
                    print(f"  Church {churches} is not scheduled")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = ChurchesPartialSolutionPrinter(
    schedule, num_churches, num_weeks, solution_limit
)

In [67]:
solver.Solve(model, solution_printer)

Solution 1
Week 0
  Church 0 is scheduled on week 0
  Church 1 is not scheduled
  Church 2 is not scheduled
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled
Week 1
  Church 0 is not scheduled
  Church 1 is scheduled on week 1
  Church 2 is not scheduled
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled
Week 2
  Church 0 is not scheduled
  Church 1 is not scheduled
  Church 2 is scheduled on week 2
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled
Week 3
  Church 0 is not scheduled
  Church 1 is not scheduled
  Church 2 is not scheduled
  Church 3 is scheduled on week 3
  Church 4 is not scheduled
  Church 5 is not scheduled
Solution 2
Week 0
  Church 0 is scheduled on week 0
  Church 1 is not scheduled
  Church 2 is not scheduled
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled
Week 1
  Church 0 is not scheduled
  Church 1 is scheduled on week 1
  Ch

4

In [68]:
schedule

{(0, 0): church_0 week_0(0..1),
 (0, 1): church_0 week_1(0..1),
 (0, 2): church_0 week_2(0..1),
 (0, 3): church_0 week_3(0..1),
 (1, 0): church_1 week_0(0..1),
 (1, 1): church_1 week_1(0..1),
 (1, 2): church_1 week_2(0..1),
 (1, 3): church_1 week_3(0..1),
 (2, 0): church_2 week_0(0..1),
 (2, 1): church_2 week_1(0..1),
 (2, 2): church_2 week_2(0..1),
 (2, 3): church_2 week_3(0..1),
 (3, 0): church_3 week_0(0..1),
 (3, 1): church_3 week_1(0..1),
 (3, 2): church_3 week_2(0..1),
 (3, 3): church_3 week_3(0..1),
 (4, 0): church_4 week_0(0..1),
 (4, 1): church_4 week_1(0..1),
 (4, 2): church_4 week_2(0..1),
 (4, 3): church_4 week_3(0..1),
 (5, 0): church_5 week_0(0..1),
 (5, 1): church_5 week_1(0..1),
 (5, 2): church_5 week_2(0..1),
 (5, 3): church_5 week_3(0..1)}

In [76]:
model

In [77]:
print(model)

variables {
  name: "church_0 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_0 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_0 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_0 week_3"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_3"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_3"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_3"
  

In [35]:
for church in all_churches:
    churchassigned = []
    for week in all_weeks:
        churchassigned.append(schedule[church, week])

In [36]:
churchassigned

[church_5 week_0(0..1),
 church_5 week_1(0..1),
 church_5 week_2(0..1),
 church_5 week_3(0..1)]